### Implementing Boyer-Moore

### Preprocessing the data for computing the:
#### 1. good_suffix
#### 2. bad_character rule

In [ ]:
import string

def z_array(s):
    """ Use Z algorithm (Gusfield theorem 1.4.1) to preprocess s """
    assert len(s) > 1
    z = [len(s)] + [0] * (len(s)-1)
    # Initial comparison of s[1:] with prefix
    for i in range(1, len(s)):
        if s[i] == s[i-1]:
            z[1] += 1
        else:
            break
    r, l = 0, 0
    if z[1] > 0:
        r, l = z[1], 1
    for k in range(2, len(s)):
        assert z[k] == 0
        if k > r:
            # Case 1
            for i in range(k, len(s)):
                if s[i] == s[i-k]:
                    z[k] += 1
                else:
                    break
            r, l = k + z[k] - 1, k
        else:
            # Case 2
            # Calculate length of beta
            nbeta = r - k + 1
            zkp = z[k - l]
            if nbeta > zkp:
                # Case 2a: Zkp wins
                z[k] = zkp
            else:
                # Case 2b: Compare characters just past r
                nmatch = 0
                for i in range(r+1, len(s)):
                    if s[i] == s[i - k]:
                        nmatch += 1
                    else:
                        break
                l, r = k, r + nmatch
                z[k] = r - k + 1
    return z


def n_array(s):
    """ Compile the N array (Gusfield theorem 2.2.2) from the Z array """
    return z_array(s[::-1])[::-1]


def big_l_prime_array(p, n):
    """ Compile L' array (Gusfield theorem 2.2.2) using p and N array.
        L'[i] = largest index j less than n such that N[j] = |P[i:]| """
    lp = [0] * len(p)
    for j in range(len(p)-1):
        i = len(p) - n[j]
        if i < len(p):
            lp[i] = j + 1
    return lp


def big_l_array(p, lp):
    """ Compile L array (Gusfield theorem 2.2.2) using p and L' array.
        L[i] = largest index j less than n such that N[j] >= |P[i:]| """
    l = [0] * len(p)
    l[1] = lp[1]
    for i in range(2, len(p)):
        l[i] = max(l[i-1], lp[i])
    return l


def small_l_prime_array(n):
    """ Compile lp' array (Gusfield theorem 2.2.4) using N array. """
    small_lp = [0] * len(n)
    for i in range(len(n)):
        if n[i] == i+1:  # prefix matching a suffix
            small_lp[len(n)-i-1] = i+1
    for i in range(len(n)-2, -1, -1):  # "smear" them out to the left
        if small_lp[i] == 0:
            small_lp[i] = small_lp[i+1]
    return small_lp


def good_suffix_table(p):
    """ Return tables needed to apply good suffix rule. """
    n = n_array(p)
    lp = big_l_prime_array(p, n)
    return lp, big_l_array(p, lp), small_l_prime_array(n)


def good_suffix_mismatch(i, big_l_prime, small_l_prime):
    """ Given a mismatch at offset i, and given L/L' and l' arrays,
        return amount to shift as determined by good suffix rule. """
    length = len(big_l_prime)
    assert i < length
    if i == length - 1:
        return 0
    i += 1  # i points to leftmost matching position of P
    if big_l_prime[i] > 0:
        return length - big_l_prime[i]
    return length - small_l_prime[i]


def good_suffix_match(small_l_prime):
    """ Given a full match of P to T, return amount to shift as
        determined by good suffix rule. """
    return len(small_l_prime) - small_l_prime[1]


def dense_bad_char_tab(p, amap):
    """ Given pattern string and list with ordered alphabet characters, create
        and return a dense bad character table.  Table is indexed by offset
        then by character. """
    tab = []
    nxt = [0] * len(amap)
    for i in range(0, len(p)):
        c = p[i]
        assert c in amap
        tab.append(nxt[:])
        nxt[amap[c]] = i+1
    return tab

### Class used for Implementing Boyer-Moore

In [ ]:
class BoyerMoore(object):
    """ Encapsulates pattern and associated Boyer-Moore preprocessing. """
    
    def __init__(self, p, alphabet='ACGT'):
        self.p = p
        self.alphabet = alphabet
        # Create map from alphabet characters to integers
        self.amap = {}
        for i in range(len(self.alphabet)):
            self.amap[self.alphabet[i]] = i
        # Make bad character rule table
        self.bad_char = dense_bad_char_tab(p, self.amap)
        # Create good suffix rule table
        _, self.big_l, self.small_l_prime = good_suffix_table(p)
    
    def bad_character_rule(self, i, c):
        """ Return # skips given by bad character rule at offset i """
        assert c in self.amap
        ci = self.amap[c]
        assert i > (self.bad_char[i][ci]-1)
        return i - (self.bad_char[i][ci]-1)
    
    def good_suffix_rule(self, i):
        """ Given a mismatch at offset i, return amount to shift
            as determined by (weak) good suffix rule. """
        length = len(self.big_l)
        assert i < length
        if i == length - 1:
            return 0
        i += 1  # i points to leftmost matching position of P
        if self.big_l[i] > 0:
            return length - self.big_l[i]
        return length - self.small_l_prime[i]
    
    def match_skip(self):
        """ Return amount to shift in case where P matches T """
        return len(self.small_l_prime) - self.small_l_prime[1]

### Testing the following (pre-processing functions):
###### 1. bad_charcter_rule function
###### 2. good_suffix_rule function
###### 3. match_skip function 

### Setting up the Text (T) and the Pattern (P)

In [ ]:
# GCTAGCTC (Text = t)
# TCAA (Pattern = p)
p = 'TCAA'

### Calling the bad character rule function

In [ ]:
# Create a Boyer-Moore object (This will do the preprocessing for that pattern)
p_bm = BoyerMoore(p)
# Call the bad character rule function (call the offset where the match occurred
# and the mismatch in t)
p_bm.bad_character_rule(2, 'T')

2

### Calling the good suffix rule

In [ ]:
# GCTAGCTC (Text = t)
# ACTA (Pattern = p)
p = 'ACTA'
# Create a Boyer-Moore object (This will do the preprocessing for that pattern)
p_bm = BoyerMoore(p)
# Call the good suffix rule (Only have to pass in where the index or the 
# mismatch occurs)
p_bm.good_suffix_rule(0) # In pattern p 'CTA' match with t, mismatch at 
                         # position 0, A with G

3

### Calling the match-skip function

In [ ]:
# ACACGCTC
# ACAC
p = 'ACAC'
# Create a Boyer-Moore object (This will do the preprocessing for that pattern)
p_bm = BoyerMoore(p)
p_bm.match_skip() # first AC in the pattern matches the second AC in the text

2

### Creating the Boyer-Moore function

In [ ]:
def boyer_moore(p, p_bm, t): # p=pattern, p_bm=Boyer-Moore Object, t=text
    i = 0
    occurrences = []
    while i < len(t) - len(p) + 1:
        # amount pattern p will move along text t for comparison can be >1
        shift = 1
        # will be updated if we find a mismatch                         
        mismatched = False 
        for j in range(len(p)-1, -1, -1): # loop through text from back to front
            if not p[j] == t[i+j]:        # character comparison
                # calling bad character rule
                skip_bc = p_bm.bad_character_rule(j, t[i+j])
                # calling good suffix rule 
                skip_gs = p_bm.good_suffix_rule(j)
                # move p along t by the max of either shift, skip_bc or skip_gs           
                shift = max(shift, skip_bc, skip_gs) 
                mismatched = True         # mismatch found
                break                     # break loop if mismatch found
        if not mismatched:                # no mismatch, exact match of text t
            # add position onto list of occurences
            occurrences.append(i)
            # still skip some using match skip function         
            skip_gs = p_bm.match_skip()
            # move p along t by the max of either shift or skip_gs   
            shift = max(shift, skip_gs)   
        i += shift                        # update our position by shift
    return occurrences                    # Return the list of occurrences

    

    

### Test the Boyer-Moore function on a simple text

In [ ]:
t = 'GCTACGATCTAGAATCTA'
p = 'TCTA'

# Create Boyer-Moore object to do pre-processing
p_bm = BoyerMoore(p)

In [ ]:
# Run Boyer-Moore function
boyer_moore(p, p_bm, t)

[7, 14]

### Check the result above is correct

In [ ]:
t[7:11]

'TCTA'

In [ ]:
t[14:18]

'TCTA'